In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("OFF")

### Reading user-artist interaction data

In [3]:
user_artist_data_path = "data/audioscrobbler/user_artist_data.txt"
raw_user_artist_data = spark.read.text(user_artist_data_path)

In [4]:
raw_user_artist_data.show(3) # user ID, artist ID, and play count

+------------------+
|             value|
+------------------+
|      1000002 1 55|
|1000002 1000006 33|
| 1000002 1000007 8|
+------------------+
only showing top 3 rows



`raw_user_artist_data` is like a list of strings. Not very useful. We parse it into a DataFrame with 3 integer-type columns.

In [5]:
raw_user_artist_data.createOrReplaceTempView("raw_user_artist_data")

query = """
    select 
        cast(split(value, ' ')[0] as int) as user,
        cast(split(value, ' ')[1] as int) as artist,
        cast(split(value, ' ')[2] as int) as count
    from raw_user_artist_data
"""
user_artist_data = spark.sql(query)

In [6]:
user_artist_data.show(3)

+-------+-------+-----+
|   user| artist|count|
+-------+-------+-----+
|1000002|      1|   55|
|1000002|1000006|   33|
|1000002|1000007|    8|
+-------+-------+-----+
only showing top 3 rows



### Reading artist aliases

In [7]:
artist_alias_path = "data/audioscrobbler/artist_alias.txt"
raw_artist_alias = spark.read.text(artist_alias_path)

raw_artist_alias.createOrReplaceTempView("raw_artist_alias")

query = """
    select 
        cast(split(value, '\t')[0] as int) as artist,
        cast(split(value, '\t')[1] as int) as alias
    from raw_artist_alias
"""
artist_alias = spark.sql(query)

In [8]:
artist_alias.show(3)

+-------+-------+
| artist|  alias|
+-------+-------+
|1092764|1000311|
|1095122|1000557|
|6708070|1007267|
+-------+-------+
only showing top 3 rows



### Prepare the training and testing data

Using `artist_alias` to replace artist IDs by their appropriate alias.

In [9]:
from pyspark.sql.functions import when, col

user_artist_data.createOrReplaceTempView("user_artist_data")
artist_alias.createOrReplaceTempView("artist_alias")

df = spark.sql("""
    select * 
    from user_artist_data 
        natural left join artist_alias;
""") 

df = df.withColumn(
    "artist", # column name
    when(col("alias").isNull(), col("artist")).otherwise(col("alias")) # when(condition, value).otherwise(value)
).drop("alias")

# could have used lateral here ig

In [10]:
df = df.sample(0.02) # only taking 2% of the data because I'm poor
train, test = df.randomSplit([8.0, 2.0])

### Training the model

In [11]:
train.cache()

DataFrame[artist: int, user: int, count: int]

In [12]:
from pyspark.ml.recommendation import ALS

als = ALS(userCol='user', itemCol='artist', ratingCol='count')
model = als.fit(train)

In [13]:
predictions = model.transform(test)
predictions.na.drop().show(5)

+------+-------+-----+----------+
|artist|   user|count|prediction|
+------+-------+-----+----------+
|     1|1000140|    4|-7.0502887|
|     1|1000534|    4|-0.7253994|
|     1|1001106|    1| 15.552921|
|     1|1001358|   66| 50.990677|
|     1|1001672|    1|-1.7063985|
+------+-------+-----+----------+
only showing top 5 rows



In [14]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="count")
evaluator.evaluate(predictions.na.drop())

117.97707169298894

### Hyperparameter tuning

In [15]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [1, 5, 10]) \
    .addGrid(als.maxIter, [5, 10]) \
    .addGrid(als.regParam, [0.05, 0.1]) \
    .build()

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator)

cv_model = cv.fit(train)
predictions = cv_model.transform(test)
evaluator.evaluate(predictions.na.drop())

277.18891965808376